# Set Covering Problem
#### AhmadReza Nopoush_______________________id:610301194

### Introduction:
The Set Covering Problem (SCP) is a NP-hard combinatorial optimization problem; so the exact algorithms, can’t find optimal 
solutions within a reasonable amount of time. ACO is meta-heuruestic algorithm wich is used for Graph problems, so if we modelize the scp with graph, we will can implement the ACO. The basic idea is to imitate the cooperative behavior of ant colonies for the search of the optimal solution.

#### libraries:
in this code we shall use numpy, because it is fast and efficient with data. also we use random, for producing and shuffling.

In [10]:
import numpy as np
import random

#### input & initialization:
before we begin, we input the data of scp from our computer. The output consists of a 2-D array called "matrix", and a list called "Cost", which represents the cost of using each set.
Each column of the matrix represents a set and each row represents a particular member. Element i is in set j if and only if matrix[i][j] = 1. otherwise matrix[i][j] = 0. The problem is to choose sets with minimum possible cost that include all members.

In [11]:
Cost = list()

#note that you can input your file by changing the address below.
with open("scpb1.txt",mode="r") as file:
    count = -3
    idx = 0
    flag = 0
    
    for l in file.readlines():
        l = l[1:]
        if count == -3:
            ij = list(l.split())
            row_count = int(ij[0])
            column_count = int(ij[1])
            matrix = np.zeros((row_count,column_count))
            count = -1
            
        elif count == -1:
            Cost += list(int(i) for i in list(l.split()))
            if len(Cost) == column_count:
                count = 0
                
        elif count == 0:
            element_repeat = int(l.split()[0])
            count = 1
            
        elif count == 1:
            jarray = [int(i) for i in l.split()]
            for i in jarray:
                matrix[idx][i-1] = 1
                flag += 1
            if flag == element_repeat:
                idx += 1
                flag = 0
                count = 0

#### Ants:
The representation of the solution, for an ant, has the form of a vector S of bits 
of size n that may take the values 0-1, n stands for the number of columns of the SCP(count of sets). If the 
column j belongs to an ant solution then the jth bit takes the value 1 otherwise it takes the 
value 0. An ant solution cost is simply calculated by: f(S)= ∑j=1..n Cost[j]*S[j]

The structure of the ant class is as follows. Ant class has three main attributes. The solution list, as described, indicates whether the jth set is present in the solution. The "order" shows the count of columns(sets) and the "cost" attribute represent the cost of the solution. As we said, the goal is to minimize the cost.

Ant class has 4 functions.  The "totalCost" function returns the cost of the solution by recieving the cost list. The "randomInsert" function, by taking a percentage as a number between 0 and 1, randomly sets that percentage of solution components equal to 1.  Later we will explain the role of this function. The "setElements" function returns the jth set as a python set. The "isCoverd" function tells us whether the solution has covered all the elements or not(there is still element i that are not in the solution).

In [12]:
class Ant:
    
    def __init__(self, order:int):
        self.order = order
        self.solution = [0 for i in range(self.order)]
        self.cost = 0
     
    #return the cost of the solution.
    def totalCost(self, CostArray:list):
        fitness = 0
        for idx in range(self.order):
            fitness += (CostArray[idx]*self.solution[idx])
        return fitness
    
    #randomly change a given percententage solution components to 1.
    def randomInsert(self, Percentage:float) -> None:
        count = int(Percentage*self.order)
        for x in range(count):
            self.solution[x] = 1
        random.shuffle(self.solution)
    
    #return Column as a python set.    
    def SetElements(self,matrix,Jth:int) -> set:
        Set = set()
        for r in range(len(matrix)):
            if matrix[r][Jth]==1:
                Set.add(r+1)
        return Set
    
    #return true of the solution covers the elements.
    def isCovered(self,matrix) -> bool:
        whole = set(range(1,len(matrix)+1))
        for j in range(self.order):
            if self.solution[j]==1:
                whole -= self.SetElements(matrix,j)
                if len(whole) == 0:
                    return True
        return False

### MMAS

ACO class is the class which we run the MMAS algorithm. MMAS is an algorithm whose base is the same as the ACO, with the difference that the amount of pheromones does not exceed a certain limit and the pheromones are updated according to the best answer found.


### Class Initialization:
1. tao: the maximum limit of pheromones.
2. pheromone: A list of lenght n(number of columns) that represents the pheromone to select that columns. Note that the initial value of pheromone is equal to tao according to MMAS.
3. card: A list of lenght n that represents the cardinality of each sets(sets are columns).
4. cost: A list of lenght n that shows the cost of selecting of each columns
5. rho: Pheromone evaporation rate per generation.
6. alpha: exponent which effects the column pheromone in column selection.
7. beta: exponent which effects the column cost in column selection.
8. generation: numbers of iteration.
9. Antnumber: numbers of ants per iteration.
10. percentage: The percentage of indexes whose value is 1 at the beginning of the algorithm.
11. rho1: exponent which effects the column pheromone in local Search.
12. rho2: exponent which effects the column pheromone in local Search.



### probability function:
This function outputs the probability distribution of selecting each column. The probability of selecting each column is obtained from the following formula:

P(j) = 0   if j∉Sk

P(j) = (Phero[j] ** alpha)*((card[j]/cost[j]) ** beta) / ∑ j=1..n (Phero[j] ** alpha)*((card[j]/cost[j])) O.W



### elimination function:
Once a covering has been established by an ant, we introduce an iterative function that eliminates the redundant columns, i.e., the one that are useless since their elements are covered by other columns of the covering. This function iterates on the columns, beginning from the last ones whose cost is more important, in order to optimize the cost.



### LocalSearch function:
The local search is applied to the last generation of ants and at the end of the search algorithm. This allows a local optimization of the found solutions. Effectively, the tests effectuated on the ants have given solutions equal to 99% of the optimal solutions.

This function is implemented according to the description in the paper. But one of the challenges of my implementation is that local search rarely implements local optimality and its cost is usually more than the cost of the input solution. I don't know if my implementation is wrong or if this local search is not compatible with MMAS.

### Run function:
The Run function executes the main MMAS algorithm. For each generation of ants, we generate a number of all-zero solutions (all solution indexes are zero) and set a percentage of its index to 1 randomly. we will enter the while loop and repeat the loop until our solution covers the set completely. In the loop, according to the probability formula inside the paper, we choose a column randomly. When we exit the loop, we execute the elimination function to remove the extra columns.  This is the end of the ant's movement. Then we select the best ant in each generation and update the phormons according to the best answer found.

In [13]:
class ACO:
    
    def __init__(self,matrix,Cost:list,Antnum:int,Generation:int,rho:float,alpha:int,beta:int,
                 perc:float,tao:float,rho1:float,rho2:float) -> None:
        self.jCount = len(matrix[0])
        self.iCount = len(matrix)
        
        #For each column j Do Phero[j] := Phero0
        self.pheromone = [tao for j in range(self.jCount)]
        
        #parameters initialization : α, β, ρ, ρ1, ρ2, ants_nb, generation_nb 
        self.tao = tao
        self.card = [int(j) for j in matrix.sum(axis = 0)]
        self.cost = Cost
        self.rho = rho
        self.alpha = alpha
        self.beta = beta
        self.generation = Generation
        self.Antnumber = Antnum
        self.percentage = perc
        self.rho1 = rho1
        self.rho2 = rho2
        
    def probibility(self, ant:Ant) -> list:
        sigma = 0
        prob_list = list()
        for j in range(ant.order):
            prob = 0
            if ant.solution[j]==0:
                prob = ((self.pheromone[j])**self.alpha + (self.card[j]/self.cost[j])**self.beta)
            prob_list.append(prob)
            sigma += prob
        for j in range(len(prob_list)):
            prob_list[j]/=sigma
        return prob_list
    
    def elimination(self,matrix, ant:Ant) -> Ant:
        oneIdx = [j for j in range(ant.order) if ant.solution[j] == 1]
        oneIdx.sort(key = lambda x : self.cost[x])
        oneIdx.reverse()
        for j in oneIdx:
            ant.solution[j] = 0
            if not ant.isCovered(matrix):
                ant.solution[j] = 1
        ant.cost = ant.totalCost(self.cost)
        return ant
    
    def UpdatePheromone(self,best:Ant) -> None:
        self.pheromone = [(1-self.rho)*i for i in self.pheromone]
        for j in range(best.order):
            if best.solution[j]==1:
                self.pheromone[j] += (1/best.cost)
                if self.pheromone[j] > self.tao:
                    self.pheromone[j] = self.tao
                    
    def LocalSearch(self,matrix, best:Ant) -> Ant:
        #define new ant
        better = Ant(best.order)
        better.solution = best.solution
        better.cost = best.cost
        
        #find E,D
        maxcost = 0
        columns = list()
        for j in range(best.order):
            if better.solution[j] == 1:
                columns.append(j)
                if self.cost[j] > maxcost:
                    maxcost = self.cost[j]
        D = int(self.rho1 * len(columns))
        E = self.rho2 * maxcost
        
        #Choose D columns to eliminate from the solution S.
        Zeros = np.random.choice(columns,D,replace=False)
        for j in Zeros:
            better.solution[j] = 0
            
        #when the covering is not yet effectuated do 
        TheSet = set()
        columns = list()
        for j in range(better.order):
            if better.solution[j]==1:
                columns.append(j)
                TheSet.update(better.SetElements(matrix,j))
                
        while len(TheSet)!=len(matrix):
            chosen = list()
            mincost = min([self.cost[j]/self.card[j] for j in range(better.order) if 
                           (better.solution[j]==0 and self.cost[j]<=E)])
            
            #record all the columns j such as cj≤E | j ∉ S.
            for j in range(better.order):
                if better.solution[j]==0 and self.cost[j] <= E:
                    if mincost == self.cost[j]/self.card[j]:
                        chosen.append(j)
                        
            #randomly choose between the recorded columns the set k that has the min of the ratio cj/cardj
            kj = int(random.choice(chosen))
            columns.append(kj)
            better.solution[kj] = 1
            TheSet.update(better.SetElements(matrix,kj))
            
            #Add k to S and eliminate
            flag_set = set()
            for j in columns:
                flag_set.update(better.SetElements(matrix,j))
            oneIdxs = columns
            oneIdxs.sort(key = lambda x : self.card[x]/self.cost[x])
            oneIdxs.reverse()
            for j in oneIdxs:
                if flag_set - better.SetElements(matrix,j) == flag_set:
                    better.solution[j] = 0
                    columns.remove(j)
        
        better.cost = better.totalCost(self.cost)
        return better
    
    def Run(self,matrix) -> Ant:
        
        #For each ant generation Do:
        for G in range(self.generation):
            
            #For each antk Do:
            Ants = [Ant(self.jCount) for i in range(self.Antnumber)]
            for a in range(self.Antnumber):
                print(a+1)
                #Choose percentage columns randomly and insert them in the partial solution of the ant.
                Ants[a].randomInsert(self.percentage)
                
                #Since the covering is not yet effectuated Do:
                while not Ants[a].isCovered(matrix):
                    
                    #Choose a column with regard to probaility function
                    ProbList = self.probibility(Ants[a])
                    chosenIndex = int(np.random.choice(Ants[a].order,1,p=ProbList))
                    
                    #Insert the chosen column in the partial solution of the ant.
                    Ants[a].solution[chosenIndex] = 1
                    
                Ants[a] = self.elimination(matrix,Ants[a])
            
            #Calculate the best solution (best_cost)    
            Ants.sort(key = lambda x : x.cost)
            print("this iteration: ", Ants[0].cost,Ants[-1].cost)
            if G==0 or Ants[0].cost < best_solution.cost:  
                best_solution = Ants[0]
                
            #Global updating of pheromone 
            self.UpdatePheromone(best_solution)
            
        #Apply the local search on the best found solution.    
        ls = self.LocalSearch(matrix,best_solution)
        if ls.cost < best_solution.cost:
            best_solution = ls
        print("the best: ",best_solution.cost)
        return best_solution

In [29]:
al = ACO(matrix,Cost,10,20,0.8,3,7,0.01,2,0.15,1.1)
b = al.Run(matrix)
b.solution

1


C:\Users\mhnop\AppData\Local\Temp\ipykernel_14592\2535791143.py:135: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  chosenIndex = int(np.random.choice(Ants[a].order,1,p=ProbList))


2
3
4
5
6
7
8
9
10
this iteration:  81 629
1
2
3
4
5
6
7
8
9
10
this iteration:  82 507
1
2
3
4
5
6
7
8
9
10
this iteration:  81 317
1
2
3
4
5
6
7
8
9
10
this iteration:  86 547
1
2
3
4
5
6
7
8
9
10
this iteration:  86 363
1
2
3
4
5
6
7
8
9
10
this iteration:  81 268
1
2
3
4
5
6
7
8
9
10
this iteration:  85 339
1
2
3
4
5
6
7
8
9
10
this iteration:  83 273
1
2
3
4
5
6
7
8
9
10
this iteration:  83 436
1
2
3
4
5
6
7
8
9
10
this iteration:  84 697
1
2
3
4
5
6
7
8
9
10
this iteration:  84 356
1
2
3
4
5
6
7
8
9
10
this iteration:  81 511
1
2
3
4
5
6
7
8
9
10
this iteration:  96 531
1
2
3
4
5
6
7
8
9
10
this iteration:  81 449
1
2
3
4
5
6
7
8
9
10
this iteration:  78 283
1
2
3
4
5
6
7
8
9
10
this iteration:  83 442
1
2
3
4
5
6
7
8
9
10
this iteration:  81 312
1
2
3
4
5
6
7
8
9
10
this iteration:  109 564
1
2
3
4
5
6
7
8
9
10
this iteration:  78 326
1
2
3
4
5
6
7
8
9
10
this iteration:  85 409
the best:  78


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


### 41 
##### with the parameters ACO(matrix, Cost, 10, 10, 0.8, 3, 5, 0.01, 2, 0.15, 1.1) we got[436, 441, 441, 442, 438, 441, 442, 436, 441, 441] in 10 times.

### 51
##### ACO(matrix,Cost,5,10,0.9,9,20,0.01,2,0.15,1.1) we got [273, 271, 274, 279, 272, 271, 269, 278]

### 54
##### with the parameters ACO(matrix,Cost, 4 , 10 , 0.9 , 1 , 5 , 0.01 , 2 , 0.15 , 1.1) we got[247, 246, 248, 247, 247, 248, 246, 247, 247, 248] in 10 timess.

### a2
##### ACO(matrix,Cost,5,20,0.8,9,4,0.01,2,0.15,1.1) we get [267, 269, 261, 260, 267] in 5 times 

### b1
##### ACO(matrix,Cost,10,20,0.8,3,7,0.01,2,0.15,1.1) we get [78, 83, 76, 81, 82] in 5 times

### My Code Problems:
It can be said that my code has 2 major problems. First, this code is not efficient for large data, so it takes a long time to get the solution. To reduce the time, the solution can be quickly converged to the optimum by increasing the rho factor, but here is the problem, when we get stuck in the optimum, we cannot escape from it, and therefore our exploration power is greatly reduced and best answer shall never found.
Maybe if we can write the ant class more efficiently or use better local search, we will get better answers.